In [1]:
import torch
import torch.nn.functional as F
import torch.nn as nn
from torch import optim
import numpy as np
import pandas as pd
import os
import random
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Subset
from PIL import Image
import albumentations as A
from tqdm import tqdm
import matplotlib.pyplot as plt
from fixpoint_utils import (
    keypoints_IoU_mult,
    get_acc, 
    get_acc_EfficentNet,
    save_checkpoint, 
    load_checkpoint, 
    get_loader_keypoint)
from Keypoint_dataset_gaussian import get_loader_keypoint_simulated
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau
from models_2 import KeypointModel, KeypointModel_with_segm, UNET, KeypointModel_3, EfficentNet

INFO:albumentations.check_version:A new version of Albumentations is available: 1.4.19 (you have 1.4.12). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
/usr/lib/python3/dist-packages/requests/__init__.py:87: RequestsDependencyWarning: urllib3 (2.2.2) or chardet (4.0.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [2]:
import wandb
import os 

os.environ['WANDB_NOTEBOOK_NAME'] =
%env WANDB_API_KEY = 

wandb.login()

wandb: WARNING WANDB_NOTEBOOK_NAME should be a path to a notebook file, couldn't find /home/alex/Documents/new try/Lung Unet/compare_models.ipynb.


env: WANDB_API_KEY=32ba2dfcae28d5229d9da5ea919130a28f8ab52c


/usr/lib/python3/dist-packages/requests/__init__.py:87: RequestsDependencyWarning: urllib3 (2.2.2) or chardet (4.0.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
wandb: Currently logged in as: alexander-ortlieb94 (ulm-university). Use `wandb login --relogin` to force relogin


True

In [3]:
class weighted_MSELoss(nn.Module):
    def __init__(self):
        super().__init__()
    def forward(self,inputs,targets,weights):
        return ((inputs - targets)**2 ) * weights

In [4]:
# HYPERPARAMETERS:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
LOGGING = False
NUM_EPOCHS = 50
random.seed(42)
torch.manual_seed(42)

In [5]:
def get_acc(model, seg_model, data_loader, loss_func, DEVICE = 'cuda:0'):
    model.eval()
    loss = 0
    IoU = 0
    with torch.no_grad():
        for image, targ in data_loader:
            img = image.to(device = DEVICE)
            mask = seg_model(img)
            pred_mask = (torch.sigmoid(mask) > 0.5)
            added_tensor = img + pred_mask
            concat_tensor = torch.cat((img, pred_mask, added_tensor), dim=1)
            target = targ.to(DEVICE)
            pred = model(concat_tensor)
            loss += loss_func(pred, target.squeeze(1))
            IoU += keypoints_IoU_mult(image, targ.squeeze(1), pred)
    model.train()
    print('Test loss:', loss.item()/len(data_loader), 'Test Box IoU:', IoU.item()/len(data_loader.dataset))
    return loss.item()/len(data_loader), IoU.item()/len(data_loader.dataset)

In [6]:
def training(i, model, seg_model, BATCH_SIZE):
    train_loader, test_loader, validation_loader = get_loader_keypoint_simulated(BATCH_SIZE)
    learning_rate = 0.001
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    loss_func = nn.MSELoss()
    min_loss = 1_000_000

    scaler = torch.cuda.amp.GradScaler()

    run = wandb.init(
                # set the wandb project where this run will be logged
                #entity = "alexander-ortlieb94",
                project = "Fixpoint detection + segmentation simulated",
                name = 'EffNet & Monai_unet',
                job_type = 'gaussian',
                
                config={
                "try no.": i
                },
    )

    for epoch in range(NUM_EPOCHS):
        loop = tqdm(train_loader)
        for batch_idx, (img, target_keypoints) in enumerate(loop):
            img = img.to(device = DEVICE)
            target_keypoints = target_keypoints.to(DEVICE)
            mask = seg_model(img)
            pred_mask = (torch.sigmoid(mask) > 0.5)
            added_tensor = img + pred_mask
            concat_tensor = torch.cat((img, pred_mask, added_tensor), dim=1)
            with torch.cuda.amp.autocast():
                predictions = model(concat_tensor) # [k,8]
                loss = loss_func(predictions, target_keypoints.squeeze(1)) #torch.mean(loss_func(predictions, target_keypoints.squeeze(1), weights))

            
            optimizer.zero_grad()
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            loop.set_postfix(loss=loss.item())

        val_loss, val_iou = get_acc(model, seg_model, validation_loader, loss_func)  #get_acc(model, validation_loader, loss_func) 
        wandb.log(
            {
                "val loss": val_loss,
                "val Box IoU": val_iou
            }
        )
       
    
    checkpoint = {
            "state_dict": model.state_dict(),
            "optimizer": optimizer.state_dict(),
            }
    save_path = f'/home/alex/Documents/new try/Data/Studie_daten/safe_states/EffNet_Own_unet_gaussian_{i}.pth.tar'
    save_checkpoint(checkpoint, save_path)
    
    test_loss, test_iou = get_acc(model, seg_model, test_loader, loss_func) 
    wandb.log(
            {
                "test loss": test_loss,
                "test Box IoU": test_iou
            }
        )
    run.finish()
    del loss, predictions, model, test_loss, test_iou, val_iou, val_loss
    torch.cuda.empty_cache() 

In [7]:
from monai.networks.nets import UNet
BATCH_SIZE = 8

for i in range(5):
            #Own_unet = UNET(in_channels=1, out_channels=1, features= [16, 32, 64, 128]).to(DEVICE)
            #load_checkpoint(Own_unet, r'/home/alex/Documents/new try/Data/Lung Unet/save_states/Own_Unet_gaussian_10.pth.tar') 
            #seg_model = UNET(in_channels=1, out_channels=1, features= [16, 32, 64, 128]).to(DEVICE)
            #load_checkpoint(seg_model, f'/home/alex/Documents/new try/Data/Lung Unet/save_states/simulate_data/Own_Unet_{i}.pth.tar')
            model = EfficentNet().to(DEVICE) #KeypointModel().to(device=DEVICE)
            load_checkpoint(model, f'/home/alex/Documents/new try/Data/Studie_daten/safe_states/EfficentNet_gaussian_{i}.pth.tar')
            Monai_unet = UNet(spatial_dims=2, in_channels=1, out_channels=1, channels=(16, 32, 64, 128), strides=(2, 2, 2), bias = False, num_res_units = 0).to(device=DEVICE)
            load_checkpoint(Monai_unet, f'/home/alex/Documents/new try/Data/Lung Unet/save_states/simulate_data/Monai_Unet_gaussian{i}.pth.tar')
            training(i, model, Monai_unet, BATCH_SIZE)
            

=> Loading checkpoint
=> Loading checkpoint


100%|██████████| 43/43 [01:05<00:00,  1.52s/it, loss=107] 


Test loss: 60.69521484375 Test Box IoU: 0.8578410736502033


100%|██████████| 43/43 [00:54<00:00,  1.26s/it, loss=35.9]


Test loss: 51.255889892578125 Test Box IoU: 0.8845180354706229


...

Test loss: 44.234170532226564 Test Box IoU: 0.8839524151527718


100%|██████████| 43/43 [00:50<00:00,  1.17s/it, loss=17.1]


Test loss: 38.12425537109375 Test Box IoU: 0.8982808361314747
=> Saving checkpoint
Test loss: 44.53085632324219 Test Box IoU: 0.8885608829864083


test Box IoU,▁
test loss,▁
val Box IoU,▅█▅▁▇▅▇▆▆▇██▇▇▇▄▇▆▇▆█▆▆█▆▇▇▇▆██▆▆▅▆▆▇▇▇▇
val loss,▄▂▃█▃▃▄▂▃▂▂▂▂▂▂▇▃▃▂▂▂▄▄▂▂▂▁▁▂▂▂▂▂▃▂▃▃▂▂▁
test Box IoU,0.88856
test loss,44.53086
val Box IoU,0.89828
val loss,38.12426


: 

# compare error of lungs to spine

In [8]:
train_loader, test_loader = get_loader_keypoint(BATCH_SIZE = 8)
n = len(test_loader)
loss_fn = nn.MSELoss()
Keyp = KeypointModel().to(DEVICE)
checkpoint_Keyp = r'/home/alex/Documents/new try/Data/Studie_daten/safe_states/Keypoint_detection_padding_0.001_1_0_6.pth.tar'
load_checkpoint(Keyp, checkpoint_Keyp)
loss_lung = 0
loss_spine = 0
for image, targ in test_loader:
    img_Keyp = image.to(device = DEVICE)
    target = targ.to(DEVICE).squeeze(1)
    pred_keyp = Keyp(img_Keyp)
    loss_spine += loss_fn(pred_keyp[:,:4], target[:,:4]).item()
    loss_lung += loss_fn(pred_keyp[:,4:], target[:,4:]).item()

print(loss_spine/n)
print(loss_lung/n)

=> Loading checkpoint
64.07782454239695
127.99258964940121
